In [ ]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

Virtual screen libraries:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Restart the notebook to strat the virtual screen:

In [ ]:
import os
os.kill(os.getpid(), 9)

In [3]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [4]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [5]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 0
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 3
Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 3
Action taken: 0
Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 3


In [6]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

In [7]:
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device


In [8]:
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
# Save the model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.6     |
|    ep_rew_mean     | -193     |
| time/              |          |
|    fps             | 2239     |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 91.6         |
|    ep_rew_mean          | -153         |
| time/                   |              |
|    fps                  | 1954         |
|    iterations           | 2            |
|    time_elapsed         | 16           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0061498787 |
|    clip_fraction        | 0.0372       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.00259     |
|    learning_r

In [9]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=268.75 +/- 19.63485389637082


In [10]:
notebook_login()
!git config --global credential.helper store

In [14]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

## TODO: Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
repo_id = 'kowalsky/LunarLander-v2'

# TODO: Define the name of the environment
env_id = 'LunarLander-v2'

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])


# TODO: Define the model architecture we used
model_architecture = "PPO"

## TODO: Define the commit message
commit_message = "Added trained LunarLander-v2"

# method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpplm916yk/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpplm916yk/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpplm916yk/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpplm916yk/-step-0-to-step-1000.mp4
ℹ Pushing repo kowalsky/LunarLander-v2 to the Hugging Face Hub


policy.pth:   0%|          | 0.00/43.3k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/87.9k [00:00<?, ?B/s]

ppo-LunarLander-v2.zip:   0%|          | 0.00/147k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_variables.pth:   0%|          | 0.00/431 [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/kowalsky/LunarLander-v2/tree/main/


'https://huggingface.co/kowalsky/LunarLander-v2/tree/main/'